In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

In [2]:
print(torch.__version__)

2.1.2+cu121


In [3]:
train_path = 'D:\\ProgPrj\\dsProjects\\gazprom-media\\ml\\train'

test_path = 'D:\\ProgPrj\\dsProjects\\gazprom-media\\ml\\test'

In [4]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')

In [5]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomApply(torch.nn.ModuleList([transforms.ColorJitter()]), p=0.25),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomRotation(degrees=(-10, 10)),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.1, value='random')
])

In [6]:

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [7]:
train_dataset = datasets.ImageFolder(train_path, train_transform)
val_dataset = datasets.ImageFolder(test_path, val_transform)

batch = 64

train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch, shuffle=False, num_workers=4, pin_memory=True)

In [8]:
print(len(train_dataset.class_to_idx))

num_of_classes = len(train_dataset.class_to_idx)

101


In [9]:
model = models.resnext101_32x8d(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_of_classes)
model = model.to(DEVICE)

C:\Users\Geo\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Geo\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [10]:
criterion = nn.CrossEntropyLoss()
#=====================================================================
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm

num_epochs = 20
train_loss_history = []
val_accuracy_history = []
val_f1_history = []

scaler = GradScaler()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    train_loader_tqdm = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')

    for inputs, labels in train_loader_tqdm:
        inputs, labels = inputs.to(DEVICE, non_blocking=True), labels.to(DEVICE, non_blocking=True)

        optimizer.zero_grad()

        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * inputs.size(0)
        train_loader_tqdm.set_postfix(loss=loss.item())

    epoch_loss = running_loss / len(train_dataset)
    train_loss_history.append(epoch_loss)

Epoch 2/20:  15%|█▍        | 173/1184 [01:01<05:11,  3.24it/s, loss=3.77]

In [ ]:
# Оценка модели на валидационном наборе данных
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(DEVICE, non_blocking=True), labels.to(DEVICE, non_blocking=True)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='macro')
val_accuracy_history.append(accuracy)
val_f1_history.append(f1)
print(f'Validation Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}')